In [1]:
import pandas as pd
rating_movies = pd.read_csv('data/ratings_movies.csv')

In [68]:
import numpy as np

In [14]:
movies = pd.read_csv('data/movies.csv')

In [2]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    
rating_movies['year_release'] = rating_movies['title'].apply(get_year_release)

In [20]:
filter = rating_movies['year_release'] == 1999
rating_movies[filter].groupby(by='movieId')['rating'].mean().sort_values()

movies[movies['movieId'] == 145951]

,movieId,title,genres
9121,145951,Bloodsport: The Dark Kumite (1999),Action|Thriller


In [22]:
filter = rating_movies['year_release'] == 2010
rating_movies[filter].groupby(by='genres')['rating'].mean().sort_values()


genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [51]:
users_genres = rating_movies.groupby(by=['userId', 'genres'])['year_release'].last()
users_genres = users_genres.reset_index()
users_genres.groupby(by='userId')['genres'].count().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [53]:
rating_movies.groupby(by='userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [76]:
filter = rating_movies['year_release'] == 2018
genres = rating_movies[filter].groupby(by='genres')['rating'].agg(['mean', 'count'])
genres[genres['count'] > 10].sort_values(by='mean', ascending=False)


,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [108]:
rating_movies['year_rating'] = pd.to_datetime(rating_movies['date']).dt.year
pt = rating_movies.pivot_table(values='rating', index='genres', columns='year_rating', fill_value=0)
pt.loc['Comedy']

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64

In [116]:
orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')

orders_products = orders.merge(products, how='left', left_on='ID товара', right_on='Product_ID')

In [129]:
filter = orders_products['Оплачен'] == 'Да'
paid_orders = orders_products[filter].copy()
paid_orders['sale'] = paid_orders['Price'] * paid_orders['Количество']
paid_orders.groupby('ID Покупателя')['sale'].sum()

ID Покупателя
1        0.0
5    13043.0
7    17096.0
8     1344.0
Name: sale, dtype: float64